In [70]:
import sys
sys.path.append("..")

import pandas as pd
from tabulate import tabulate
import talib

In [71]:
print("Loading stock data from CSV files")
tickers = ["AAPL", "AMZN", "GOOG", "META", "MSFT", "NVDA"]

stock_data = {}

for t in tickers:
    df = pd.read_csv(f"../data/finance_data/{t}.csv")
    stock_data[t] = df


Loading stock data from CSV files


In [72]:
print("Checking for missing values in each dataset:")
for ticker, df in stock_data.items():
    print(f"\nSummary statistics for {ticker}:")
    missing_summary = df.isna().sum().reset_index()
    missing_summary.columns = ['Column', 'Missing Values']
    print(tabulate(missing_summary, headers="keys", tablefmt="psql", showindex=False))

Checking for missing values in each dataset:

Summary statistics for AAPL:
+----------+------------------+
| Column   |   Missing Values |
|----------+------------------|
| Date     |                0 |
| Close    |                0 |
| High     |                0 |
| Low      |                0 |
| Open     |                0 |
| Volume   |                0 |
+----------+------------------+

Summary statistics for AMZN:
+----------+------------------+
| Column   |   Missing Values |
|----------+------------------|
| Date     |                0 |
| Close    |                0 |
| High     |                0 |
| Low      |                0 |
| Open     |                0 |
| Volume   |                0 |
+----------+------------------+

Summary statistics for GOOG:
+----------+------------------+
| Column   |   Missing Values |
|----------+------------------|
| Date     |                0 |
| Close    |                0 |
| High     |                0 |
| Low      |                0 |
|

In [73]:
print("Cleaning and aligning datasets:")

summary = []

date_sets = [set(pd.to_datetime(df["Date"])) for df in stock_data.values()]
common_dates = set.intersection(*date_sets)

for ticker, df in stock_data.items():
    before_rows = df.shape[0]

    # Convert Date column to datetime
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

    # Drop duplicates
    df.drop_duplicates(subset="Date", inplace=True)

    # Filter to common dates
    df = df[df["Date"].isin(common_dates)].copy()

    # Sort by date
    df.sort_values("Date", inplace=True)

    # Adjust index
    df.reset_index(drop=True, inplace=True)

    # Get after rows
    after_rows = df.shape[0]

    # Save back to dictionary
    stock_data[ticker] = df

    summary.append(
        {
            "Ticker": ticker,
            "Rows Before": before_rows,
            "Rows After": after_rows,
            "Duplicates Removed": before_rows - after_rows,
            "Date dtype": df["Date"].dtype,
        }
    )

print(tabulate(summary, headers="keys", tablefmt="psql"))

Cleaning and aligning datasets:
+----------+---------------+--------------+----------------------+----------------+
| Ticker   |   Rows Before |   Rows After |   Duplicates Removed | Date dtype     |
|----------+---------------+--------------+----------------------+----------------|
| AAPL     |          3774 |         2923 |                  851 | datetime64[ns] |
| AMZN     |          3774 |         2923 |                  851 | datetime64[ns] |
| GOOG     |          3774 |         2923 |                  851 | datetime64[ns] |
| META     |          2923 |         2923 |                    0 | datetime64[ns] |
| MSFT     |          3774 |         2923 |                  851 | datetime64[ns] |
| NVDA     |          3774 |         2923 |                  851 | datetime64[ns] |
+----------+---------------+--------------+----------------------+----------------+


In [75]:
print("Analysis with TA-Lib")

indicators_data = {}
summary = []

for ticker, df in stock_data.items():
    temp = df.copy()

    temp["SMA_20"] = talib.SMA(temp["Close"], timeperiod=20)

    temp["EMA_20"] = talib.EMA(temp["Close"], timeperiod=20)

    temp["RSI_14"] = talib.RSI(temp["Close"], timeperiod=14)

    macd, macd_signal, macd_hist = talib.MACD(
        temp["Close"], fastperiod=12, slowperiod=26, signalperiod=9
    )
    temp["MACD"] = macd
    temp["MACD_Signal"] = macd_signal
    temp["MACD_Hist"] = macd_hist

    indicators_data[ticker] = temp
    last_row = temp.iloc[-1]
    summary.append(
        {
            "Ticker": ticker,
            "Total Rows": temp.shape[0],
            "SMA_20": round(last_row["SMA_20"], 2),
            "EMA_20": round(last_row["EMA_20"], 2),
            "RSI_14": round(last_row["RSI_14"], 2),
            "MACD": round(last_row["MACD"], 4),
            "MACD_Signal": round(last_row["MACD_Signal"], 4),
            "MACD_Hist": round(last_row["MACD_Hist"], 4),
        }
    )

print(tabulate(summary, headers="keys", tablefmt="psql"))

Analysis with TA-Lib
+----------+--------------+----------+----------+----------+--------+---------------+-------------+
| Ticker   |   Total Rows |   SMA_20 |   EMA_20 |   RSI_14 |   MACD |   MACD_Signal |   MACD_Hist |
|----------+--------------+----------+----------+----------+--------+---------------+-------------|
| AAPL     |         2923 |   192.49 |   191.43 |    51.12 | 1.5595 |        2.4246 |     -0.8651 |
| AMZN     |         2923 |   149.82 |   150.21 |    59.01 | 2.782  |        2.923  |     -0.141  |
| GOOG     |         2923 |   135.98 |   137.41 |    59.31 | 1.8428 |        1.3059 |      0.5369 |
| META     |         2923 |   336.87 |   341.66 |    64.3  | 8.1933 |        6.6935 |      1.4998 |
| MSFT     |         2923 |   367.79 |   367.79 |    57.95 | 2.6536 |        3.1496 |     -0.496  |
| NVDA     |         2923 |    48.05 |    48.49 |    58.31 | 0.6975 |        0.6352 |      0.0624 |
+----------+--------------+----------+----------+----------+--------+----------